##### ВЫГРУЗКА БИБЛИОТЕК

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.callbacks import EarlyStopping

##### ПЕРВИЧНАЯ ВЫГРУЗКА И ОБРАБОТКА ДАННЫХ

In [2]:
images = np.load('/kaggle/input/mipt-dl-captcha/data/images.npy')
images = images/255

labels = np.load('/kaggle/input/mipt-dl-captcha/data/labels.npy')
labels = to_categorical(labels, num_classes=26)

In [3]:
# Разделение данных на обучающую, валидационную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

##### РАБОТА С МОДЕЛЬЮ

In [4]:
# Создание модели CNN
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(26, activation='softmax'))

c:\Users\pront\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
# Компиляция модели
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Аугментация данных
datagen = ImageDataGenerator(
                            rotation_range=10,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            zoom_range=0.1,
                            )
datagen.fit(X_train)


In [6]:
# Обучение модели с использованием аугментации данных
batch_size = 55 # для accuracy 0.79 в kaggle - 50
epochs = 40     # для accuracy 0.79 в kaggle - 40

cnn_model = model.fit(
                    datagen.flow(X_train, y_train, batch_size=batch_size),
                    epochs = epochs,
                    validation_data = (X_val, y_val),
                    steps_per_epoch = len(X_train)
                    )

# Оценка модели
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Точность на тестовой выборке:', accuracy)

images_sub = np.load('/kaggle/input/mipt-dl-captcha/data/images_sub.npy')

# Предсказание меток классов
predictions = model.predict(images_sub)
predicted_labels = np.argmax(predictions, axis=1)

# Выгрузка результатов в папку
submission = pd.DataFrame({'Id': range(len(predicted_labels)), 'Category': predicted_labels})
submission.to_csv('submission_cnn_{0}.csv'.format('1'), index=False)

Epoch 1/40
    4/12800 ━━━━━━━━━━━━━━━━━━━━ 3:42 17ms/step - accuracy: 0.0174 - loss: 3.2656        

c:\Users\pront\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  231/12800 ━━━━━━━━━━━━━━━━━━━━ 3:16 16ms/step - accuracy: 0.0490 - loss: 3.2506

c:\Users\pront\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


12800/12800 ━━━━━━━━━━━━━━━━━━━━ 5s 307us/step - accuracy: 0.0485 - loss: 3.2373 - val_accuracy: 0.0525 - val_loss: 3.1808
Epoch 2/40
12800/12800 ━━━━━━━━━━━━━━━━━━━━ 4s 298us/step - accuracy: 0.0912 - loss: 3.1112 - val_accuracy: 0.2188 - val_loss: 2.8208
Epoch 3/40
12800/12800 ━━━━━━━━━━━━━━━━━━━━ 4s 300us/step - accuracy: 0.2093 - loss: 2.7000 - val_accuracy: 0.4319 - val_loss: 2.0698
Epoch 4/40
12800/12800 ━━━━━━━━━━━━━━━━━━━━ 4s 297us/step - accuracy: 0.3077 - loss: 2.3074 - val_accuracy: 0.5328 - val_loss: 1.6072
Epoch 5/40
12800/12800 ━━━━━━━━━━━━━━━━━━━━ 4s 296us/step - accuracy: 0.3779 - loss: 2.0671 - val_accuracy: 0.5878 - val_loss: 1.4742
Epoch 6/40
12800/12800 ━━━━━━━━━━━━━━━━━━━━ 4s 296us/step - accuracy: 0.4252 - loss: 1.8984 - val_accuracy: 0.6484 - val_loss: 1.2642
Epoch 7/40
12800/12800 ━━━━━━━━━━━━━━━━━━━━ 4s 294us/step - accuracy: 0.4525 - loss: 1.7877 - val_accuracy: 0.6734 - val_loss: 1.1490
Epoch 8/40
12800/12800 ━━━━━━━━━━━━━━━━━━━━ 4s 294us/step - accuracy: 0.4

In [7]:
# Повторное обучение <-- в итоге ведёт к переобучению и снижению результатов
# cnn_model = model.fit(
#                     datagen.flow(X_train, y_train, batch_size=batch_size),
#                     epochs = epochs,
#                     validation_data = (X_val, y_val),
#                     steps_per_epoch = len(X_train)
#                     )

# # Оценка модели
# loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
# print('Точность на тестовой выборке:', accuracy)

# images_sub = np.load('images_sub.npy')

# # Предсказание меток классов
# predictions = model.predict(images_sub)
# predicted_labels = np.argmax(predictions, axis=1)

# submission = pd.DataFrame({'Id': range(len(predicted_labels)), 'Category': predicted_labels})
# submission.to_csv('submission_cnn_{0}.csv'.format('2'), index=False)